## Fine-tune Llama for SUD

In [ ]:
!pip3 install -q -U accelerate==0.23.0 peft==0.5.0 bitsandbytes==0.41.1 transformers==4.31 trl==0.7.2
!pip3 install xformers
!pip3 install tensorboardX

## Import libraries

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

## Data Loading


In [35]:
# Read data from CSV file into a pandas DataFrame
filename = "all.csv"
df = pd.read_csv(filename)
# Select only the "input" and "labels" columns from the DataFrame
df = df[["input", "labels"]]


In [36]:
df

,input,labels
0,i hate a bitch that think they crazy,offensive
1,people from tucson are the expes on tucson onc...,neither
2,damn good thing the left is sending their best...,hate
3,nothing this person lost one of the dearest pp...,neither
4,lol your rd grade education is impressive to y...,hate
...,...,...
101175,gets real interesting here friends when you re...,neither
101176,new age spiritual leader in the aicle about ne...,neither
101177,medical treatments for nationals of us should ...,neither
101178,gimme a break these are hot dogs not frankfuer...,neither


In [37]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
unique_values = df['labels'].unique()

# Print the unique values
print(unique_values)

['offensive' 'neither' 'hate' 'severe_toxic' 'toxic' 'insult' 'profane'
 'obscene' 'identity_hate' 'threat']


In [38]:
# Assuming df is your DataFrame and 'column_name' is the name of the column
df['labels'] = df['labels'].replace({'severe_toxic': 'severe', 'identity_hate': 'identity', "insult": "ins", "offensive": "off", "profane": "prof", "obscene": "obsc", "toxic": "to"})

# Now 'severe_toxic' should be replaced with 'severe toxic' and 'intentity_hate' with 'identity hate'


In [39]:
unique_values = df['labels'].unique()

print(unique_values)

['off' 'neither' 'hate' 'severe' 'to' 'ins' 'prof' 'obsc' 'identity'
 'threat']


## Data Splitting and Prompt Generation

In [51]:
X_train = list()
X_test = list()
# Split the data into training, testing, and evaluation sets for each sentiment label
for labels in ["off", "neither", "hate", "severe", "to", "ins", "prof", "obsc", "identity", "threat"]:
    train, test  = train_test_split(df[df.labels==labels], 
                                    train_size=90,
                                    test_size=10, 
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

# Concatenate and shuffle the training and testing sets
X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

# Select evaluation samples from the remaining data
eval_idx = [idx for idx in df.index if idx not in list(train.index) + list(test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('labels', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))

# Reset index of the training set
X_train = X_train.reset_index(drop=True)

# Function to generate prompts for training and evaluation data
def generate_prompt(data_point):
    return f"""
            Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, 
            and return the answer as the corresponding label:
            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat". 
            Make sure to give the whole label as an answer.
            [{data_point["input"]}] = {data_point["labels"]}
            """.strip()

# Function to generate prompts for testing data
def generate_test_prompt(data_point):
    return f"""
            Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, 
            and return the answer as the corresponding label:
            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".
            Make sure to give the whole label as an answer.
            [{data_point["input"]}] = """.strip()

# Convert the prompts into DataFrames for training, evaluation, and testing
X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), 
                       columns=["input"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), 
                      columns=["input"])

y_true = X_test.labels
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["input"])

# Create datasets from the generated prompts
train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [52]:
labels

'threat'

## Evaluation function

In [53]:
def evaluate(y_true, y_pred):
    # Define labels and their corresponding numeric mappings
    labels = ["off", "neither", "hate", "severe", "to", "ins", "prof", "obsc", "identity", "threat"]
    mapping = {"off": 9, "neither": 8, "hate": 7, "severe" : 6, "to": 5, "ins": 4, "prof": 3, "obsc": 2, "identity": 1, "threat": 0}
    
    # Function to map labels to numeric values
    def map_func(x):
        return mapping.get(x, 1)
    
    # Apply mapping to true and predicted labels
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    print('\nConfusion Matrix:')
    print(conf_matrix)



## Model Configuration and Initialization

In [54]:
#model_name = "../input/llama-2/pytorch/7b-hf/1"
model_name = "meta-llama/Llama-2-7b-hf"
#model_name = "meta-llama/Llama-2-13b-hf"
# Define the data type for model computation
compute_dtype = getattr(torch, "float16")

# Configure BitsAndBytes quantization parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Initialize the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
)

# Initialize the model with quantization settings
model.config.use_cache = False
model.config.pretraining_tp = 1

# Initialize the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
# Set padding token and side for the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.91s/it]


## Prediction Function


In [55]:
def predict(test, model, tokenizer):
    """
    Make predictions using a text generation model.

    Parameters:
        test (pd.DataFrame): DataFrame containing test data.
        model: Pre-trained text generation model.
        tokenizer: Tokenizer for the model.

    Returns:
        list: Predicted labels for each input.
    """    
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["input"]
        # Create a text generation pipeline
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 1, 
                        temperature = 0.0,
                       )
        # Generate text based on the prompt
        result = pipe(prompt)
        print(result)
        answer = result[0]['generated_text'].split("=")[-1]
        print(answer)
        # Map the generated text to sentiment labels
        if "off" in answer:
            y_pred.append("off")
        elif "hate" in answer:
            y_pred.append("hate")
        elif "severe" in answer:
            y_pred.append("severe")
        elif "to" in answer:
            y_pred.append("to")
        elif "ins" in answer:
            y_pred.append("ins")
        elif "prof" in answer:
            y_pred.append("prof")
        elif "obsc" in answer:
            y_pred.append("obsc")
        elif "identity" in answer:
            y_pred.append("identity")
        elif "threat" in answer:
            y_pred.append("threat")
        else:
            y_pred.append("neither")
    return y_pred

In [56]:
y_pred = predict(test, model, tokenizer)


  2%|▏         | 2/100 [00:00<00:12,  7.89it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [ik i look good and all but dam bitches stop making it obv your staring at me loll] = "'}]
 "
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [a bitch who cant ride

  4%|▍         | 4/100 [00:00<00:10,  8.84it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [the metoo movement is just an extension of the democrats kkk amp the hate group antifa its just another form of lynching those who oppose their agenda metoo people jump on ban wagons in the name of justice to perpetuate mass bullying of those who threaten to expose them] = "'}]
 "
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "ne

  6%|▌         | 6/100 [00:00<00:10,  9.32it/s]

...

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [thats it remember last time you fucked with me the dildo your ear all that blood youve given me no choice but to repeat the punishment this time it will be a incher] = "'}]
 "
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            

 98%|█████████▊| 98/100 [00:10<00:00,  8.26it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [must be tali ihantala where finland germany sweden and norway joined forces and killed every f cking russian war criminal that mad insane dictator stalin sent once and for all those ruthless russkie thieves stole the karelia from finland give it back you vodka drinking bastards or there will be trouble] = "'}]
 "
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding

100%|██████████| 100/100 [00:11<00:00,  8.93it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [my thoughts youre a fucking piece of shit die and burn in hell] = "'}]
 "
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [hey you fascist swine youre gonna get ra

In [57]:
evaluate(y_true, y_pred)


Accuracy: 0.100
Accuracy for label 0: 0.000
Accuracy for label 1: 0.000
Accuracy for label 2: 0.000
Accuracy for label 3: 0.000
Accuracy for label 4: 0.000
Accuracy for label 5: 0.000
Accuracy for label 6: 0.000
Accuracy for label 7: 0.000
Accuracy for label 8: 1.000
Accuracy for label 9: 0.000

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        10
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00        10
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        10
           8       0.10      1.00      0.18        10
           9       0.00      0.00      0.00        10

    accuracy                           0.10       100
   macro avg       0.01      0

## LoRA Model Configuration

In [58]:
# LoRA Model Configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
# Training Arguments Configuration
training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

# Trainer Initialization
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="input",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=1024,
)

Map: 100%|██████████| 500/500 [00:00<00:00, 2678.87 examples/s]


## Model training

In [59]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,0.733600,0.827192
2,0.609600,0.810914
2,0.694900,0.806901
4,0.583500,0.809294
4,0.693200,0.811752


## Model Predictions

In [60]:
y_pred = predict(test, model, tokenizer)
evaluate(y_true, y_pred)

  2%|▏         | 2/100 [00:00<00:18,  5.41it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [ik i look good and all but dam bitches stop making it obv your staring at me loll] = off'}]
 off
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [a bitch who cant 

  4%|▍         | 4/100 [00:00<00:16,  5.76it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [the metoo movement is just an extension of the democrats kkk amp the hate group antifa its just another form of lynching those who oppose their agenda metoo people jump on ban wagons in the name of justice to perpetuate mass bullying of those who threaten to expose them] = hate'}]
 hate
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" 

  6%|▌         | 6/100 [00:01<00:15,  5.98it/s]

...

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [fuck you slakr die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipedia die wikipe

 99%|█████████▉| 99/100 [00:17<00:00,  5.40it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [hi were bp hello were gonna kill the gulf and yo and spill oil on your body and light ya up and shoot ya dead bodie cause were evil] = threat'}]
 threat
[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the w

100%|██████████| 100/100 [00:17<00:00,  5.64it/s]

[{'generated_text': 'Categorize the tweet enclosed in square brackets to determine if it is off, or neither, or hate, or severe, or to, or ins, or prof, or obsc, or identity, or threat, \n            and return the answer as the corresponding label:\n            "off" or "neither" or "hate" or "severe" or "to" or "ins" or "prof" or "obsc" or "identity" or "threat".\n            Make sure to give the whole label as an answer.\n            [hey you fascist swine youre gonna get raped in the ass nice job being jimbos bitch you pathetic piece of shit] = threat'}]
 threat
Accuracy: 0.470
Accuracy for label 0: 0.800
Accuracy for label 1: 0.600
Accuracy for label 2: 0.200
Accuracy for label 3: 0.500
Accuracy for label 4: 0.200
Accuracy for label 5: 0.700
Accuracy for label 6: 0.300
Accuracy for label 7: 0.300
Accuracy for label 8: 0.600
Accuracy for label 9: 0.500

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        